In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
try:
    os.makedirs('../outputs')
except Exception as e:
    print(f'Directory Exists!')
logdir = r'/kaggle/outputs/'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install randaugment
# # %cd c:\users\user\appdata\local\programs\python\python38\lib\site-packages 
# from randaugment import RandAugment
# # %cd C:\XAI

In [ ]:
!pip install split-folders

In [ ]:
import os
import cv2
logdir = r'/kaggle/outputs/'
import torch
import PIL 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import torch.nn.functional as F 
import warnings,copy, random, time, glob, math, torch, argparse
from torch import nn,optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from IPython.display import FileLink
from zipfile import ZipFile
from pathlib import Path
import splitfolders
writer = SummaryWriter(log_dir=logdir, flush_secs=10)
torch.manual_seed(3407)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

In [ ]:
def load_from_checkpoint(PATH):
    checkpoint = torch.load(PATH)
    last_epoch = checkpoint['epoch']
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']
    best_test_acc = checkpoint['acc']
    last_lr = checkpoint['last_lr']
    return model, optimizer, last_epoch, best_test_acc, last_lr
def save_to_checkpoint(epoch, model, optimizer, test_acc, last_lr, PATH):
    torch.save({'epoch': epoch, 'model': model,'optimizer': optimizer,'acc': test_acc, 'last_lr': last_lr}, PATH)

In [ ]:
def PrintImagefromDataloader(iterator, i):
    dataiter = iter(iterator)
    images, labels = dataiter.next()
    plt.imshow(np.transpose(images[i].cpu().detach().numpy(), (1, 2, 0)))
    plt.show()
def GetImage(iterator):
#     warnings.filterwarnings("ignore", category=UserWarning) 
    samples = iter(iterator)
    data, label = samples.next()
    return data
def plot_graph(paramlist, picname):
    plt.figure(figsize=(12,3))
    plt.plot(paramlist)
    plt.title(label=f'Best Acc : {max(paramlist)*100}%')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.savefig(picname, dpi=300, bbox_inches='tight')

In [ ]:
def calc_acc(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs
def train_one_epoch(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for i, (x, y) in enumerate(iterator):
        x = x.to(device, non_blocking=True)
        if isinstance(y, (tuple, list)):
            y1, y2, lam = y
            y = (y1.to(device, non_blocking=True), y2.to(device, non_blocking=True), lam)
        else:
            y = y.to(device, non_blocking=True)
        optimizer.zero_grad() 
        y_pred = model(x) 
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        # Old - >
        acc = calc_acc(y_pred, y)
        # acc = cutmix_calc_acc(outputs=y_pred, targets=y, data=x) #review
        epoch_loss += loss.item()
        epoch_acc += acc #check  
    return epoch_loss / len(iterator), epoch_acc / len(iterator), optimizer.param_groups[0]["lr"]
def test_one_epoch(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for i, (x, y) in enumerate(iterator):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            y_pred= model(x)
            loss = criterion(y_pred, y)
            acc = calc_acc(y_pred=y_pred, y=y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()  
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
def train_loop(last_epoch, epochs, best_test_acc, model, trainiterator, testiterator, traincriteron, 
               testcriteron, optimizer, 
#                scheduler, 
               device, SAVEPATH):
    history, logs = [[], [], [], []], []
    
    for epoch in range(last_epoch+1, last_epoch+epochs+1):
#         warnings.filterwarnings("ignore", category=UserWarning)
        start_time = time.monotonic()
        train_loss, train_acc, last_lr = train_one_epoch(model=model, iterator=trainiterator, criterion=traincriteron, 
                                                    optimizer=optimizer, device=device)
        test_loss, test_acc = test_one_epoch(model=model, iterator=testiterator, criterion=testcriteron, device=device)
#         scheduler.step()
        end_time = time.monotonic()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        log = f'Epoch: {epoch:04d} ({epoch_mins}m {epoch_secs}s) | Train Loss: {train_loss:.5f}, Test Loss: {test_loss:.5f} | Train Acc: {train_acc*100:.5f}, Test Acc: {test_acc*100:.5f}'
        print(log)
#         print(f'Epoch: {epoch:04d} ({epoch_mins}m {epoch_secs}s), lr = {optimizer.param_groups[0]["lr"]} | Train Loss: {train_loss:.5f}, Test Loss: {test_loss:.5f} | Train Acc: {train_acc*100:.5f}, Test Acc: {test_acc*100:.5f}')
        
        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("Acc/train", train_acc, epoch)
        writer.add_scalar("Loss/Valid", test_loss, epoch)
        writer.add_scalar("Acc/Valid", test_acc, epoch)
        history[0].append(train_loss) 
        history[1].append(train_acc)
        history[2].append(test_loss)
        history[3].append(test_acc)
        logs.append(log)
        if test_acc>best_test_acc:
            best_test_acc = test_acc
            save_to_checkpoint(epoch=epoch, model=model, optimizer=optimizer, test_acc=test_acc, PATH=SAVEPATH, last_lr=last_lr)
    return history, logs

In [ ]:
def get_linlayer_idx(model):
    offset = len(model._modules['features']) + len([model._modules['avgpool']]) # for 1 avgpool layer
    indices = [] 
    for i, layer in enumerate(model._modules['classifier']):
        if isinstance(layer, nn.Linear):
            indices.append(offset+i)
    return indices
def linear_to_conv(layers):
    newlayers = [] 
    for i, layer in enumerate(layers):
        if isinstance(layer, nn.Linear):
            newlayer = None
            # layer -> [m, n, k, k], weight -> [n, m, k, k] 
            kernel_size = 7 if i==0 else 1
            m, n = layer.weight.shape[1]//(kernel_size**2), layer.weight.shape[0]
            newlayer = nn.Conv2d(in_channels=m, out_channels=n, kernel_size=kernel_size) 
            newlayer.weight = nn.Parameter(data=layer.weight.reshape(n, m, kernel_size, kernel_size))
            newlayer.bias = nn.Parameter(layer.bias)
            newlayers += [newlayer]
        else:
            newlayers += [layer]
    return newlayers
def newlayer(layer, func):
    layer = copy.deepcopy(layer)
    try:
        layer.weight = torch.nn.Parameter(func(layer.weight))
    except AttributeError:
        pass
    try:
        layer.bias = torch.nn.Parameter(func(layer.bias))
    except AttributeError:
        pass
    return layer
def get_rho_incr(n):
    if n>=31:
        rho = lambda p : p 
        incr = lambda z : z+1e-9
    elif 17<=n<=30:
        rho = lambda p : p 
        # z : z + 1e-9 + gamma*root(mean(z^2)) ;;; rms(z)
        incr = lambda z : z+1e-9+0.25*((z**2).mean()**.5).data  
    elif n<=16:
        # gamma=0.25 ; p : p (1+gamma)
        # weight tensor is clamped to get only positive val
        rho = lambda p : p + 0.25*p.clamp(min=0) 
        incr = lambda z : z+1e-9  
    return rho, incr
def apply_lrp(model, image):
        # Step 1 -> 
        image = torch.unsqueeze(image, dim=0)
        layers = list(model._modules['features']) + [model._modules['avgpool']] + linear_to_conv(list(model._modules['classifier']))
        linlayer_indices = get_linlayer_idx(model)
        n = len(layers) 
        activations = [image] + [None]*n 
        
        # Step 2 ->
        for i in range(n):
#                 print(f'{i}, {layers[i]}')
                if i in linlayer_indices and i==32:
                        activations[i] = activations[i].reshape((1, 512, 7, 7))
                activation = layers[i].forward(activations[i])
                if isinstance(layers[i], torch.nn.modules.pooling.AdaptiveAvgPool2d):
                        activation = torch.flatten(activation, start_dim=1)
                activations[i+1] = activation
                
        # Step 3 ->
        one_hot_out = []
        output_activation = activations[-1].detach().cpu().numpy()
        maxval = output_activation.max()
        for val in output_activation[0]:
#                 print(val)
                if val==maxval:
                        one_hot_out.append(val)
                else:
                        one_hot_out.append(0.0)
        try:
            activations[-1] = torch.FloatTensor([one_hot_out]).to(device)
        except TypeError: 
            pass    
        # Step 4 ->
        relevances = [None]*n + [activations[-1]]
        for i in range(0, n)[::-1]:
                curr_layer = layers[i]
#                 print(f'{i}, {curr_layer}')
                if isinstance(curr_layer, torch.nn.MaxPool2d):
                        layers[i] = torch.nn.AvgPool2d(kernel_size=2)
                        curr_layer = layers[i]
                if isinstance(curr_layer, torch.nn.Conv2d) or isinstance(curr_layer, torch.nn.AvgPool2d) or isinstance(curr_layer, torch.nn.Linear):
                        activations[i] = activations[i].data.requires_grad_(True)
                        rho_func, incr_func = get_rho_incr(i)
                        # Step 1
                        z = incr_func(newlayer(layer=layers[i], func=rho_func).forward(activations[i]))
                        # Step 2
                        s = (relevances[i+1]/z)
                        # Step 3
                        (z*s.data).sum().backward() ;
                        c = activations[i].grad
                        # Step 4
                        relevances[i] = (activations[i]*c).data 
                else:
                        relevances[i] = relevances[i+1]
        return relevances[0]

In [ ]:
datadir  = '../input/sarscov2-ctscan-dataset'
splitfolders.ratio(datadir, output="output", seed=1337, ratio=(.8, 0.2)) 

In [ ]:
device = torch.device("cuda:0")
# %% Load data


TRAIN_ROOT = "./output/train"
TEST_ROOT = "./output/val"
train_data = datasets.ImageFolder(root=TRAIN_ROOT)
test_data = datasets.ImageFolder(root=TRAIN_ROOT)

# %% Prepare data for pretrained model
# transforms = transforms.Compose([transforms.Resize((255,255)), transforms.ToTensor()])
train_transforms = transforms.Compose([transforms.Resize((256, 256)),
                                       transforms.RandomRotation(degrees=(-20, +20)),              
                                       transforms.CenterCrop(size=224), 
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(), 
#                                        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
                                      ])


train_dataset = datasets.ImageFolder(root=TRAIN_ROOT, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=TEST_ROOT, transform=test_transforms)

#train_dataset[0][0].permute(1,2,0)
# %% Create data loaders
batch_size = 128
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

print(f'Image shape = [{GetImage(trainloader).shape}]')
print(f'Classes : {train_data.classes}')
print(f'Image from Trainloader')
PrintImagefromDataloader(trainloader, i=0)
print(f'Image from Testloader')
PrintImagefromDataloader(testloader, i=0)   

In [ ]:
EPOCHS = 60
datasetname = f'covid-ctscan'
modelname = f'vgg16'
SAVEPATH = f"/kaggle/outputs/{datasetname}-{modelname}.pt"
last_epoch, best_test_acc = 0, 0.0
num_classes = len(train_data.classes)
model = models.vgg16(pretrained=True)
model.classifier[6].out_features = num_classes
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()
hist, logs = train_loop(last_epoch=last_epoch, epochs=EPOCHS, best_test_acc=best_test_acc, model=model, 
                        trainiterator=trainloader, testiterator=testloader,
                        traincriteron=criterion, testcriteron=criterion, 
                        optimizer=optimizer, device=device, SAVEPATH=SAVEPATH)
writer.flush()
# train_losses, train_accs, test_losses, test_accs = hist[0], hist[1], hist[2], hist[3]transforms

In [ ]:
%cd /kaggle/outputs
picname = f'{datasetname}-{modelname}-({last_epoch+1}-{last_epoch+EPOCHS})-acc-{int(max(hist[3])*100000)}.jpg'
plot_graph(hist[3], picname)
with open("train_losses.txt", "w") as output:
    output.write(str(hist[0]))
with open("train_accs.txt", "w") as output:
    output.write(str(hist[1]))
with open("test_losses.txt", "w") as output:
    output.write(str(hist[2]))
with open("test_accs.txt", "w") as output:
    output.write(str(hist[3]))
with open("logs.txt", "w") as output:
    output.write(str(logs))
%cd /kaggle/working
zipname = f'{datasetname}-{modelname}-({last_epoch+1}-{last_epoch+EPOCHS})-acc-{int(max(hist[3])*100000)}.zip'
with ZipFile(zipname, 'w') as zipf:
    for name in glob.glob('/kaggle/outputs/*'):
        zipf.write(name)
display(FileLink(zipname))

In [ ]:
# img = cv2.imread('castle.jpg')
# img = np.array(img)[..., ::-1] ## rotating axis -> rgb to bgr 
# img = img/255 # Shape : [224, 224, 3]
# # img = img[np.newaxis] ## Adds a dimension; Shape: [224, 224, 3] -> [1, 224, 224, 3]
# img = img.transpose([2, 0, 1]) ## Shape: [1, 224, 224, 3] -> [1, 3, 224, 224]
# img.shape

In [ ]:
inputs, labels = next(iter(testloader))
inputs = inputs.to(device)
labels = labels.numpy()
outputs = model(inputs).max(1).indices.detach().cpu().numpy()
comparison = pd.DataFrame()
print("Batch accuracy: ", (labels==outputs).sum()/len(labels))
comparison["labels"] = labels
comparison["outputs"] = outputs
print(comparison)
# len(inputs[:])
for i in range(127):
    pred_lebel = None
    classes = ['COVID', 'non-COVID']
#     print(f'{i}, {classes[labels[i]]}')
#     plt.imshow(inputs[i].permute(1,2,0).detach().cpu().numpy())
#     plt.show()

In [ ]:
# # visualize steps
# n= len(layers)
# acts = [torch.tensor(2)] + [None]*n
# print(acts)
# acts_last = torch.tensor([1, 2, 3, 4, 5])
# out = acts_last.detach().cpu().numpy()
# onehot_out = []
# for i in out:
#     if i == out.max():
#         onehot_out.append(i)
#     else:
#         onehot_out.append(0)
# acts_last = torch.FloatTensor([onehot_out])
# print()
# print(acts_last)
# print()
# rel = [None]*n + [acts_last]
# print(rel)

In [ ]:
# %%
# Calculate relevances for first image in this test batch
for i in range(0, 20):
    image_relevances = apply_lrp(model, inputs[i])
    image_relevances = image_relevances.permute(0,2,3,1).detach().cpu().numpy()[0]
    image_relevances = np.interp(image_relevances, (image_relevances.min(),image_relevances.max()), (0, 1))
    # Show relevances
    pred_label = list(test_dataset.class_to_idx.keys())[list(test_dataset.class_to_idx.values()).index(labels[i])]

    if outputs[i] == labels[i]:
        print("Groundtruth for this image: ", pred_label)
        # Plot images next to each other
        plt.axis('off')
        plt.suptitle(f'Image[{i}], label={pred_label}')
        plt.subplot(1,2,1)
        plt.imshow(image_relevances[:,:,0], cmap="seismic")
        plt.subplot(1,2,2)
        plt.imshow(inputs[i].permute(1,2,0).detach().cpu().numpy())
        plt.savefig(f'Covid-CTscan Image[{i}], label={pred_label}.jpg', dpi=500)
        plt.show()
    else:
        print("This image is not classified correctly.")